In [96]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV

import torch
import math
import os
# print(os.listdir("../input"))
plt.style.use('ggplot')
# Let's first import all the things we are gonna need for this task
import torch
import time 
import torch.nn as nn
import numpy as np
import random 
import torch.nn.functional as F
from torch_scatter import scatter
from torch_geometric.nn import MessagePassing
import torch_geometric.utils as U
# torch_geometric only used to load the Cora dataset
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader
from torch_geometric.data.data import Data 
import torch_geometric.utils as U
import torch
from sklearn.metrics import roc_auc_score
from torch_geometric.datasets import TUDataset

In [97]:
import torch; print(torch.__version__)

2.0.0+cu118


filepath_dict = {'yelp':   'C:/Users/---/Desktop/GDL project/sentiment labelled sentences/sentiment labelled sentences/yelp_labelled.txt',
                 'amazon': 'C:/Users/---/Desktop/GDL project/sentiment labelled sentences/sentiment labelled sentences/amazon_cells_labelled.txt',
                 'imdb':   'C:/Users/---/Desktop/GDL project/sentiment labelled sentences/sentiment labelled sentences/imdb_labelled.txt'}

df_list = []
for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath, names=['sentence', 'label'], sep='\t')
    df['source'] = source
    df_list.append(df)

In [6]:
df = pd.concat(df_list)

In [7]:
df.head()

,sentence,label,source
0,Wow... Loved this place.,1,yelp
1,Crust is not good.,0,yelp
2,Not tasty and the texture was just nasty.,0,yelp
3,Stopped by during the late May bank holiday of...,1,yelp
4,The selection on the menu was great and so wer...,1,yelp


In [8]:
from transformers import BertModel
model_name = "bert-base-uncased"
model = BertModel.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(model_name)

In [12]:
@torch.no_grad()
def bert_encode(sentences):
    toks = tokenizer.encode(sentences,return_tensors="pt")
    return model(toks[:,0:min(512,toks.shape[1])]).last_hidden_state

In [18]:
i=0

In [13]:
def reg_encode(x):
    global i
    i+=1
    if i%50==0:
        print(i)
    return bert_encode(x)

In [17]:
df = pd.concat((df[df["label"]==1][0:200],df[df["label"]==0][0:200]))

In [20]:
df["embs"] = df["sentence"].apply(reg_encode)

50
100
150
200
250
300
350
400


In [28]:
df.iloc[0]["embs"]

tensor([[[ 0.3339,  0.1434,  0.1591,  ..., -0.3110,  0.3732, -0.0155],
         [ 1.1797,  0.3336,  0.1367,  ..., -0.0649,  1.2361, -0.7014],
         [ 0.2771, -0.9117,  0.7017,  ...,  0.4588,  1.2517, -0.8002],
         ...,
         [ 0.3405,  0.5115,  0.2238,  ..., -0.3224,  0.0122, -0.1842],
         [ 0.1437, -0.3846,  0.2762,  ...,  0.6781,  0.4854, -0.6457],
         [ 0.8028,  0.1606, -0.2214,  ...,  0.1330, -0.3779, -0.3285]]])

In [25]:
def get_positional_embeddings(sequence_length, d): #"i" in paper corresponds to j - i.e. along dimension of size d
    result = torch.ones(sequence_length, d) #pos in paper refers to which token - i.e. varying from 1 to 50
    for i in range(sequence_length):
      for j in range(d):
        if j%2==0:
          result[i,j] = math.sin(i/10000**(j/d))
        else:
          result[i,j] = math.cos(i/10000**((j-1)/d))
    return result

In [33]:
j=0

In [37]:
def pos_encode(tens):
    dim = tens.shape[2]
    global j
    j+=1
    if j%50==0:
        print(j)
    data = tens
    pos = torch.unsqueeze(get_positional_embeddings(data.shape[1],data.shape[2]),0)
    return data + pos

In [38]:
df["embs"] = df["embs"].apply(pos_encode)

50
100
150
200
250
300
350
400


In [41]:
df["embs"].iloc[0].shape

torch.Size([1, 10, 768])

In [56]:
df = df.sample(frac=1)

In [57]:
df.head()

,sentence,label,source,embs
202,The warm beer didn't help.,0,yelp,"[[[tensor(0.0100), tensor(1.1372), tensor(-0.2..."
124,We thought you'd have to venture further away ...,1,yelp,"[[[tensor(0.1013), tensor(0.6968), tensor(-0.3..."
307,"As much as I'd like to go back, I can't get pa...",0,yelp,"[[[tensor(0.3022), tensor(1.1677), tensor(0.11..."
9,A great touch.,1,yelp,"[[[tensor(-0.3115), tensor(0.8831), tensor(-0...."
179,We literally sat there for 20 minutes with no ...,0,yelp,"[[[tensor(0.4016), tensor(1.1729), tensor(-0.3..."


In [66]:
Xs = df["embs"]

In [67]:
ys = df["label"]

In [68]:
X_train, X_test = Xs[0:180].reset_index(drop = True), Xs[180:200].reset_index(drop = True)

In [69]:
y_train, y_test = ys[0:180].reset_index(drop = True), ys[180:200].reset_index(drop = True)

In [84]:
torch.Tensor(y_train[0])

tensor([])

In [121]:
import torch
import os
import typing
import torch_geometric

import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.datasets as datasets
import torch_geometric.utils as U

from torch_geometric.nn import GCNConv, GAT

In [104]:
# please **DO NOT** modify  any part of the following code  in this cell 
batch_size = 4
A = []
X = []
Y = []
for idx in X_train.index:
    X_shape = X_train[idx][0].shape[0]
    adj_matrix = torch.ones((X_shape,X_shape))
    A.append(adj_matrix)
    X.append(X_train[idx][0])
    Y.append(torch.Tensor([y_train[idx]]).long())

In [105]:
print(A[0]) #ADJ matrices of varying size
print(X[0]) #node features (constant width of 7)
print(Y[0]) #graph classification (constant width 1)

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
tensor([[ 0.0100,  1.1372, -0.2214,  ...,  0.7831,  0.3334,  1.3654],
        [ 1.2660,  0.4434,  0.2629,  ...,  0.8593,  0.4246,  1.3981],
        [ 0.4131,  0.5356,  1.6886,  ...,  0.8573,  0.0506,  1.3298],
        ...,
        [ 0.5188,  1.5060,  0.8718,  ...,  0.5778, -0.5539,  0.3206],
        [ 1.8264,  0.0636,  0.6661,  ...,  1.2603, -0.4277,  0.4958],
        [ 0.8171, -0.5199,  0.5623,  ...,  0.4885, -0.2245,  0.9365]])
tensor([0])


In [88]:
for i in X:
  print((i.shape[0],i.shape[1]))

(10, 768)
(28, 768)
(29, 768)
(6, 768)
(18, 768)
(11, 768)
(15, 768)
(5, 768)
(10, 768)
(21, 768)
(7, 768)
(17, 768)
(25, 768)
(16, 768)
(15, 768)
(24, 768)
(13, 768)
(10, 768)
(11, 768)
(13, 768)
(21, 768)
(10, 768)
(19, 768)
(5, 768)
(14, 768)
(16, 768)
(10, 768)
(18, 768)
(40, 768)
(17, 768)
(7, 768)
(21, 768)
(23, 768)
(13, 768)
(8, 768)
(10, 768)
(23, 768)
(14, 768)
(16, 768)
(22, 768)
(23, 768)
(12, 768)
(11, 768)
(9, 768)
(13, 768)
(7, 768)
(12, 768)
(16, 768)
(18, 768)
(26, 768)
(26, 768)
(22, 768)
(21, 768)
(27, 768)
(6, 768)
(22, 768)
(14, 768)
(7, 768)
(7, 768)
(8, 768)
(13, 768)
(20, 768)
(11, 768)
(12, 768)
(14, 768)
(24, 768)
(24, 768)
(8, 768)
(17, 768)
(12, 768)
(5, 768)
(14, 768)
(28, 768)
(8, 768)
(32, 768)
(11, 768)
(19, 768)
(26, 768)
(16, 768)
(7, 768)
(12, 768)
(22, 768)
(21, 768)
(7, 768)
(22, 768)
(7, 768)
(18, 768)
(6, 768)
(7, 768)
(10, 768)
(29, 768)
(25, 768)
(34, 768)
(14, 768)
(7, 768)
(27, 768)
(22, 768)
(20, 768)
(25, 768)
(10, 768)
(8, 768)
(16, 768)
(1

In [106]:
from scipy.linalg import lapack
import math
def graph_mini_batch(adj_matrix_list,  x_list,  y_list, batch_size=64):
    assert(len(adj_matrix_list)==len(x_list) and len(x_list)==len(y_list))
    length_left=len(x_list)
    iters_needed = math.ceil(length_left/batch_size)

    for i in range(iters_needed):
        if length_left>=batch_size:
            this_batch=batch_size
        else:
            this_batch=length_left
        length_left -= this_batch
        #this_batch tells us how many things we're going to stick together now
        start_index = i*batch_size
        adj_round = adj_matrix_list[start_index:start_index+this_batch]
        x_round = x_list[start_index:start_index+this_batch]
        y_round = y_list[start_index:start_index+this_batch] #blah_round vars hold list of relevant tensors for this round of iter
        sizes = [x.shape[0] for x in x_round]
        batch_bits = []
        for j in range(this_batch):
            left_sum = sum(sizes[0:j])
            right_sum= sum(sizes[j+1:this_batch])
            tens = adj_round[j] # tens is the tensor at position j to stick bits onto the ends of
            tens_size = tens.shape[0]
            #MATHS: need dim0 to refer to size of tens, dim1 to refer to left and right sum
            left_block = torch.zeros(tens_size,left_sum)
            right_block = torch.zeros(tens_size,right_sum)
#            print(f"left dim: {(tens_size,left_sum)}, right dim: {(tens_size,right_sum)}, middle dim: {tens_size}, block dim: {tens_size+left_sum+right_sum}")
            adj_round[j] = torch.cat((left_block,tens,right_block),dim=1)
            batch_bit = torch.ones(tens_size)
            batch_bit = torch.mul(batch_bit,j+1)
            batch_bits.append(batch_bit)
        #stuck stuff onto adj_round tensors
        A_B = torch.cat(adj_round)
        X_B = torch.cat(x_round)
        Y_B = torch.cat(y_round)
        Batch = torch.cat(batch_bits)
        yield (A_B,X_B,Y_B,Batch)

     # Implement the function here

In [107]:
# Test your batch function here 
for adj_matrix, x, y, batch in graph_mini_batch(A, X, Y, batch_size):
     print(adj_matrix.size())
     print(x.size())
     print(y.size())
     print(batch.size())

torch.Size([73, 73])
torch.Size([73, 768])
torch.Size([4])
torch.Size([73])
torch.Size([49, 49])
torch.Size([49, 768])
torch.Size([4])
torch.Size([49])
torch.Size([55, 55])
torch.Size([55, 768])
torch.Size([4])
torch.Size([55])
torch.Size([80, 80])
torch.Size([80, 768])
torch.Size([4])
torch.Size([80])
torch.Size([47, 47])
torch.Size([47, 768])
torch.Size([4])
torch.Size([47])
torch.Size([55, 55])
torch.Size([55, 768])
torch.Size([4])
torch.Size([55])
torch.Size([58, 58])
torch.Size([58, 768])
torch.Size([4])
torch.Size([58])
torch.Size([85, 85])
torch.Size([85, 768])
torch.Size([4])
torch.Size([85])
torch.Size([54, 54])
torch.Size([54, 768])
torch.Size([4])
torch.Size([54])
torch.Size([75, 75])
torch.Size([75, 768])
torch.Size([4])
torch.Size([75])
torch.Size([55, 55])
torch.Size([55, 768])
torch.Size([4])
torch.Size([55])
torch.Size([48, 48])
torch.Size([48, 768])
torch.Size([4])
torch.Size([48])
torch.Size([92, 92])
torch.Size([92, 768])
torch.Size([4])
torch.Size([92])
torch.Size([

In [108]:
def global_sum_pool(x, batch):
    return scatter(x,batch.to(torch.int64).add(-1),0)
""" batch_size = int(torch.max(batch).item())
    index = batch.to(torch.int64).add(-1)
    index = torch.reshape(index,(index.shape[0],1))
    index = torch.cat([index]*7,1)
    size = (batch_size,x.shape[1])
    writeTensor = torch.zeros(size)
    writeTensor.scatter(0,index,x)
    return writeTensor"""
   # Implement the function here


' batch_size = int(torch.max(batch).item())\n    index = batch.to(torch.int64).add(-1)\n    index = torch.reshape(index,(index.shape[0],1))\n    index = torch.cat([index]*7,1)\n    size = (batch_size,x.shape[1])\n    writeTensor = torch.zeros(size)\n    writeTensor.scatter(0,index,x)\n    return writeTensor'

In [109]:
# Test your pooling function, assuming you are given a mini-batch of node features and a batch vector
for _, x, _ , batch in graph_mini_batch(A, X, Y, batch_size):
      sum_graph_rep =  global_sum_pool(x, batch)
      print(sum_graph_rep.size())

torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])


In [110]:
class GINConv(MessagePassing):
    def __init__(self, emb_dim):
        '''
            emb_dim (int): node embedding dimensionality
        '''
        super(GINConv, self).__init__(aggr = "add")

        self.mlp = torch.nn.Sequential(torch.nn.Linear(emb_dim, 2*emb_dim), torch.nn.BatchNorm1d(2*emb_dim), torch.nn.ReLU(), torch.nn.Linear(2*emb_dim, emb_dim))
        self.eps = torch.nn.Parameter(torch.Tensor([0]))

    def forward(self, x, edge_index):
        out = self.mlp((1 + self.eps) *x + self.propagate(edge_index, x=x))
        return out

    def message(self, x_j):
        return F.relu(x_j)

    def update(self, aggr_out):
        return aggr_out


### GNN to generate node embedding
class GIN(torch.nn.Module):
    """
    Output:
        node representations
    """
    def __init__(self, num_layer, emb_dim, hidden_dim, drop_ratio = 0, JK = "last", residual = False):
        '''
            emb_dim (int): node embedding dimensionality
            num_layer (int): number of GNN message passing layers
        '''
        super(GIN, self).__init__()
        self.num_layer = num_layer
        self.drop_ratio = drop_ratio
        self.JK = JK
        ### add residual connection or not
        self.residual = residual
        if self.num_layer < 2:
            raise ValueError("Number of GNN layers must be greater than 1.")

        self.embed = torch.nn.Linear(emb_dim, hidden_dim)
        self.convs = torch.nn.ModuleList()
        self.batch_norms = torch.nn.ModuleList()
        for layer in range(num_layer):
            self.convs.append(GINConv(hidden_dim))
            self.batch_norms.append(torch.nn.BatchNorm1d(hidden_dim))

    def forward(self, x, edge_index):
        h_list = [self.embed(x)]
        for layer in range(self.num_layer):
            h = self.convs[layer](h_list[layer], edge_index)
            h = self.batch_norms[layer](h)
            if layer == self.num_layer - 1:
                #remove relu for the last layer
                h = F.dropout(h, self.drop_ratio, training = self.training)
            else:
                h = F.dropout(F.relu(h), self.drop_ratio, training = self.training)

            if self.residual:
                h += h_list[layer]

            h_list.append(h)
        ### Different implementations of Jk-concat
        if self.JK == "last":
            node_representation = h_list[-1]
        elif self.JK == "sum":
            node_representation = 0
            for layer in range(self.num_layer + 1):
                node_representation += h_list[layer]

        return node_representation

In [115]:
class GCN(nn.Module):
  def __init__(
      self,
      input_dim: int,
      hid_dim: int,
#      n_classes: int,
      n_layers: int,
      dropout_ratio: float = 0.3):
    super(GCN, self).__init__()

    """
    Args:
      input_dim: input feature dimension
      hid_dim: hidden feature dimension
      n_classes: number of target classes
      n_layers: number of layers
      dropout_ratio: dropout_ratio
    """
    ## ------ Begin Solution ------ ##
    self.n_layers = n_layers
    if n_layers == 0:
        self.model = nn.ModuleList(
            [nn.Linear(input_dim,hid_dim),nn.ReLU(),nn.Dropout(dropout_ratio)])
        #assuming we just do a single-layer network
    elif n_layers == 1:
        self.model = nn.ModuleList(
            [GCNConv(input_dim,hid_dim),nn.ReLU(),nn.Dropout(dropout_ratio)])
    else:
        self.model = nn.ModuleList([GCNConv(input_dim,hid_dim),nn.ReLU(), nn.Dropout(dropout_ratio)]+
                              [GCNConv(hid_dim,hid_dim),nn.ReLU(),nn.Dropout(dropout_ratio)]*(n_layers-1))

    ## ------ End Solution ------ ##

  def forward(self, X, A) -> torch.Tensor:
    ## ------ Begin Solution ------ ##
    b = self.generate_node_embeddings(X,A)
    classifier = self.model[-1]
    b = classifier.forward(b)
    return b
    ## ------ End Solution ------ ##

  def generate_node_embeddings(self, X, A) -> torch.Tensor:

    b = X
    for i,l in enumerate(self.model):
        #print(i)#sanity checking

        if self.n_layers == 0 and i >= 3:#break pre-classification in 0-layer case
            break

        if i >= 3*self.n_layers and self.n_layers>0:#break pre classification
            break

        if i%3==0 and self.n_layers !=0:#if non-ReLU / dropout layer
            assert(type(l)==GCNConv)
            #print(f"b and A shapes: {b.shape}, {A.shape}")
            b = l.forward(b,A)

        else:#ReLU / dropout/n_layer=0
            b = l.forward(b)
            
    return b

  def param_init(self):
    ## ------ Begin Solution ------ ##
    """placeholder for refreshing parameters"""
    print("fired up and ready to serve")
    ## ------ End Solution ------ ##

In [123]:
from torch_scatter import scatter
from torch_geometric.utils import softmax as sparse_softmax

In [111]:
params = {
    "input_features": 768,
    "hidden_features": 50,
    "num_layers": 3,
    "learning_rate": 1e-4,
    "weight_decay": 0,
    "num_epochs": 100,
    "num_classes": 2,
    "batch_size": 63
}


In [232]:
begin_time = time.time()

#model = GIN(params["num_layers"], params["input_features"], params["hidden_features"])
model = GCN(params["input_features"], params["hidden_features"], params["num_layers"], 0)
#model = GAT( params["input_features"], params["hidden_features"],  params["num_layers"])
#GIN num_layer, emb_dim, hidden_dim, drop_ratio = 0
#GCN input_dim: int, hid_dim: int, n_classes: int, n_layers: int, dropout_ratio
#GAT in_channels: int, hidden_channels: int, num_layers: int,

pooling = global_sum_pool
graph_pred_linear = torch.nn.Linear(params["hidden_features"], params["num_classes"])
loss_fn = nn.CrossEntropyLoss()
model_param_group = [{"params": model.parameters(), "lr": params["learning_rate"]}]
if graph_pred_linear is not None:
        model_param_group.append(
            {"params": graph_pred_linear.parameters(), "lr": params["learning_rate"]}
        )

optimizer = torch.optim.AdamW(model_param_group,
                                lr=params["learning_rate"],
                                weight_decay=params["weight_decay"])
for epoch in range(params["num_epochs"]):
  model.train()
  for adj_matrix, x, y, batch in graph_mini_batch(A, X, Y, params["batch_size"]):
      optimizer.zero_grad()
      edge_index = U.dense_to_sparse(adj_matrix)[0]
      nodes = model(x, edge_index)
      graph_reps = pooling(nodes, batch)
      pred = graph_pred_linear(graph_reps)
      loss = loss_fn(pred, y)
      loss.backward()
      optimizer.step()
  if epoch % 10 == 0:
      model.eval()
      correct = 0
      total_num = 0 
      for adj_matrix, x, y, batch in graph_mini_batch(A, X, Y, params["batch_size"]):
        optimizer.zero_grad()
        edge_index = U.dense_to_sparse(adj_matrix)[0]
        nodes = model(x, edge_index)
        graph_reps = pooling(nodes, batch)
        pred = graph_pred_linear(graph_reps)
        correct += (pred.argmax(dim=-1) == y).sum()
        total_num += len(y)
      print("epoch={}, loss={}, accuracy={}".format(epoch, loss.item(), correct/total_num))
print("time={}".format(time.time()-begin_time))

epoch=0, loss=13.568131446838379, accuracy=0.3425000011920929
epoch=10, loss=0.8209672570228577, accuracy=0.4025000035762787
epoch=20, loss=0.636753499507904, accuracy=0.48500001430511475
epoch=30, loss=0.5477164387702942, accuracy=0.550000011920929
epoch=40, loss=0.47693154215812683, accuracy=0.6075000166893005
epoch=50, loss=0.41676872968673706, accuracy=0.6524999737739563
epoch=60, loss=0.36808204650878906, accuracy=0.7049999833106995
epoch=70, loss=0.3258868157863617, accuracy=0.7475000023841858
epoch=80, loss=0.2872539758682251, accuracy=0.7825000286102295
epoch=90, loss=0.25425317883491516, accuracy=0.8224999904632568
time=350.7140531539917


AGAIN! ON THE HARDER TASK!

In [96]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV

import torch
import math
import os
# print(os.listdir("../input"))
plt.style.use('ggplot')
# Let's first import all the things we are gonna need for this task
import torch
import time 
import torch.nn as nn
import numpy as np
import random 
import torch.nn.functional as F
from torch_scatter import scatter
from torch_geometric.nn import MessagePassing
import torch_geometric.utils as U
# torch_geometric only used to load the Cora dataset
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader
from torch_geometric.data.data import Data 
import torch_geometric.utils as U
import torch
from sklearn.metrics import roc_auc_score
from torch_geometric.datasets import TUDataset

In [97]:
import torch; print(torch.__version__)

2.0.0+cu118


In [124]:
data = pd.read_csv("C:/Users/---/Desktop/GDL project/yelp.csv")

In [125]:
data.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [126]:
def get_words(strr):
    words = strr.replace(".", " ")
    word_list = words.split(" ")
    return len(word_list)

In [127]:
#CREATING A NEW COLUMN IN THE DATASET FOR THE NUMBER OF WORDS IN THE REVIEW
data['length'] = data['text'].apply(get_words)
data.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny,length
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0,171
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0,274
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0,17
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0,79
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0,102


In [128]:
data["maxl"] = data["text"].apply(lambda x: max(map(len,x.split('.'))))
data.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny,length,maxl
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0,171,135
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0,274,201
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0,17,56
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0,79,172
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0,102,279


In [129]:
# CLASSIFICATION
data_classes = data[(data['stars']==1) | (data['stars']==3) | (data['stars']==5)]
data_classes.head()
print(data_classes.shape)

(5547, 12)


In [130]:
data_classes = data_classes[data_classes["maxl"]<100]
data_classes = data_classes[data_classes["length"]<200]

In [131]:
data_classes

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny,length,maxl
9,nMHhuYan8e3cONo3PornJA,2010-08-11,jJAIXA46pU1swYyRCdfXtQ,5,Nobuo shows his unique talents with everything...,review,sUNkXg8-KFtCMQDV6zRzQg,0,1,0,38,58
10,AsSCv0q_BWqIe3mX2JqsOQ,2010-06-16,E11jzpKz9Kw5K7fuARWfRw,5,The oldish man who owns the store is as sweet ...,review,-OMlS6yWkYjVldNhC31wYg,1,3,1,59,71
11,e9nN4XxjdHj4qtKCOPq_vg,2011-10-21,3rPt0LxF7rgmEUrznoH22w,5,Wonderful Vietnamese sandwich shoppe. Their ba...,review,C1rHp3dmepNea7XiouwB6Q,1,1,0,63,99
21,tdcjXyFLMKAsvRhURNOkCg,2011-06-28,LmuKVFh03Uz318VKnUWrxA,5,This place shouldn't even be reviewed - becaus...,review,YN3ZLOdg8kpnfbVcIhuEZA,1,1,2,25,97
45,qB-qsaSnhbHCt18_AN4Quw,2011-12-21,1FvrC35rTJ6BWFvRog7tuA,3,Everything was nice. The ice cream was delicio...,review,66PQJEHC0tCWGMI4V9KT-Q,0,0,0,45,95
...,...,...,...,...,...,...,...,...,...,...,...,...
9957,62F17L8z4Q4S7U_TayuDBA,2010-03-13,oTm0bBYcbgoMPJloZUpUwQ,5,Standard Mexican fare - but quite delicious. ...,review,TnTkd3MKoIOKqrsPDzhiog,0,0,0,23,45
9970,R6aazv8FB-6BeanY3ag8kw,2009-09-26,gP17ykqduf3AlewSaRb61w,5,This place is super cute lunch joint. I had t...,review,mtoKqaQjGPWEc5YZbrYV9w,0,0,0,96,98
9987,r-a-Cn9hxdEnYTtVTB5bMQ,2012-04-07,j9HwZZoBBmJgOlqDSuJcxg,1,The food is delicious. The service: discrimi...,review,toPtsUtYoRB-5-ThrOy2Fg,0,0,0,38,92
9989,mQUC-ATrFuMQSaDQb93Pug,2011-10-01,ta2P9joJqeFB8BzFp-AzjA,5,Great food and service! Country food at its best!,review,fKaO8fR1IAcfvZb6cBrs2w,0,1,0,9,49


In [132]:
stval2 = data_classes.groupby('stars')
stval2.count()

,business_id,date,review_id,text,type,user_id,cool,useful,funny,length,maxl
stars,,,,,,,,,,,
1,159,159,159,159,159,159,159,159,159,159,159
3,324,324,324,324,324,324,324,324,324,324,324
5,874,874,874,874,874,874,874,874,874,874,874


In [133]:
data_train = pd.concat([data_classes[data_classes["stars"]==i][0:150] for i in (1,3,5)]).reset_index(drop=True)

In [145]:
data_train = data_train[["text", "stars"]]

In [146]:
def list_encode(x):
    global i
    i+=1
    if i%50==0:
        print(i)
    return list(map(bert_encode, list( map( lambda x: x+".",x.split(".") ) ) )  )

In [147]:
def reg_encode(x):
    global i
    i+=1
    if i%50==0:
        print(i)
    return bert_encode(x)

In [148]:
from transformers import BertModel
model_name = "bert-base-uncased"
model = BertModel.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [149]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(model_name)

In [150]:
@torch.no_grad()
def bert_encode(sentences):
    toks = tokenizer.encode(sentences,return_tensors="pt")
    return model(toks[:,0:min(512,toks.shape[1])]).last_hidden_state

In [152]:
i=0

data_train["1D_embs"] = data_train["text"].apply(reg_encode)

data_train["2D_embs"] = data_train["text"].apply(list_encode)

In [160]:
data_train.iloc[0]["1D_embs"].shape

torch.Size([1, 50, 768])

In [174]:
data_train.iloc[0]["2D_embs"][0].shape

torch.Size([1, 17, 768])

In [175]:
def get_positional_embeddings(sequence_length, d): #"i" in paper corresponds to j - i.e. along dimension of size d
    result = torch.ones(sequence_length, d) #pos in paper refers to which token - i.e. varying from 1 to 50
    for i in range(sequence_length):
      for j in range(d):
        if j%2==0:
          result[i,j] = math.sin(i/10000**(j/d))
        else:
          result[i,j] = math.cos(i/10000**((j-1)/d))
    return result

In [176]:
j=0

In [178]:
def pos_encode(tens):
    global j
    j+=1
    if j%50==0:
        print(j)
    data = tens
    pos = torch.unsqueeze(get_positional_embeddings(data.shape[1],data.shape[2]),0)
    return data + pos

data_train["1D_embs"] = data_train["1D_embs"].apply(pos_encode)

In [180]:
data_train["1D_embs"].iloc[0].shape

torch.Size([1, 50, 768])

In [183]:
def pos_encode_2D(lst):
    """lst is list of tens. We convert to tens with 2D encoding"""
    list_len = len(lst)
    dim = lst[0].shape[-1]
    lengths = list(map(lambda x: x.shape[1],lst))#get lengths of each sentence
    newlst=[]
    assert dim%2 ==0
    dim = dim//2
    part_1 = get_positional_embeddings(list_len,dim)

    for i, elem in enumerate(lst):#doing dual encoding
        sublist_len = lengths[i]
        part_2 = get_positional_embeddings(sublist_len,dim)
        pos = torch.cat((part_1[i].expand(sublist_len,-1),part_2),1)
        newlst.append(elem+torch.unsqueeze(pos,0))
    return torch.cat(newlst,1)


data_train["2D_embs"] = data_train["2D_embs"].apply(pos_encode_2D)

In [187]:
data_train["2D_embs"].iloc[0].shape

torch.Size([1, 59, 768])

In [188]:
data_train.head()

,text,stars,1D_embs,2D_embs
0,Been here for many years but have seen a recen...,1,"[[[tensor(0.1476), tensor(0.8259), tensor(0.16...","[[[tensor(-0.0979), tensor(1.1479), tensor(0.0..."
1,This place is not there anymore.,1,"[[[tensor(0.2692), tensor(1.2510), tensor(-0.0...","[[[tensor(0.2692), tensor(1.2510), tensor(-0.0..."
2,I really don't get why everyone is raving this...,1,"[[[tensor(-0.0412), tensor(1.1868), tensor(-0....","[[[tensor(0.4508), tensor(1.3685), tensor(0.05..."
3,just say no.\n\nthis place is truly awful. \nt...,1,"[[[tensor(0.2390), tensor(1.0299), tensor(0.09...","[[[tensor(0.0316), tensor(0.9073), tensor(-0.3..."
4,"1 star for service, but the food is not ok :( ...",1,"[[[tensor(0.1576), tensor(0.8589), tensor(0.39...","[[[tensor(0.2363), tensor(0.8499), tensor(0.44..."


In [189]:
data_train = data_train.sample(frac=1)

In [190]:
data_train.head()

,text,stars,1D_embs,2D_embs
300,Nobuo shows his unique talents with everything...,5,"[[[tensor(-0.0046), tensor(0.9747), tensor(-0....","[[[tensor(-0.3308), tensor(1.2500), tensor(-0...."
76,Filthy place. Bad tasteless food. Rude waitr...,1,"[[[tensor(0.1414), tensor(1.3518), tensor(0.27...","[[[tensor(-0.1618), tensor(1.5723), tensor(-0...."
189,The Choice Hotel chain continues to be my pref...,3,"[[[tensor(-0.0424), tensor(0.9262), tensor(0.1...","[[[tensor(0.0532), tensor(1.2390), tensor(0.49..."
378,"I love pita Jungle! Everytime I am in town, I ...",5,"[[[tensor(-0.0387), tensor(0.8588), tensor(0.0...","[[[tensor(0.0577), tensor(1.2070), tensor(0.06..."
75,What ever you do don't waste your money here. ...,1,"[[[tensor(0.0339), tensor(1.0096), tensor(0.10...","[[[tensor(0.1874), tensor(1.2468), tensor(-0.3..."


In [192]:
Xs_2D = data_train["2D_embs"]

In [191]:
Xs_1D = data_train["1D_embs"]

In [193]:
ys = data_train["stars"]

In [194]:
X_train_2D, X_test_2D = Xs_2D[0:400].reset_index(drop = True), Xs_2D[400:450].reset_index(drop = True)

In [195]:
X_train_1D, X_test_1D = Xs_1D[0:400].reset_index(drop = True), Xs_1D[400:450].reset_index(drop = True)

In [196]:
y_train, y_test = ys[0:400].reset_index(drop = True), ys[400:450].reset_index(drop = True)

In [237]:
import torch
import os
import typing
import torch_geometric

import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.datasets as datasets
import torch_geometric.utils as U

from torch_geometric.nn import GCNConv, GAT, GIN

1D first

In [267]:
# please **DO NOT** modify  any part of the following code  in this cell 
batch_size = 4
A = []
X = []
Y = []
for idx in X_train_1D.index:
    X_shape = X_train_1D[idx][0].shape[0]
    adj_matrix = torch.ones((X_shape,X_shape))
    A.append(adj_matrix)
    X.append(X_train_1D[idx][0])
    Y.append(torch.Tensor([y_train[idx]//2]).long())

In [268]:
print(A[0]) #ADJ matrices of varying size
print(X[0]) #node features (constant width of 7)
print(Y[0]) #graph classification (constant width 1)

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])
tensor([[-0.0046,  0.9747, -0.1743,  ...,  0.8152,  0.5598,  1.4117],
        [-0.2956, -1.1227,  0.7439,  ...,  1.0108,  0.0295,  0.5101],
        [ 0.1561, -0.1065,  1.6898,  ...,  0.9411, -0.1688,  1.3309],
        ...,
        [ 1.3903, -1.1008,  1.1143,  ...,  0.6025, -0.0719,  0.9730],
        [-0.2650, -1.3188,  0.2608,  ...,  1.2182,  0.0592,  0.3329],
        [-0.4040, -0.3474, -0.5885,  ...,  1.1200, -0.1039,  0.5291]])
tensor([2])


In [269]:
for i in X:
  print((i.shape[0],i.shape[1]))

(43, 768)
(19, 768)
(55, 768)
(72, 768)
(115, 768)
(20, 768)
(88, 768)
(61, 768)
(28, 768)
(22, 768)
(187, 768)
(31, 768)
(104, 768)
(38, 768)
(93, 768)
(15, 768)
(10, 768)
(9, 768)
(36, 768)
(11, 768)
(57, 768)
(19, 768)
(55, 768)
(30, 768)
(70, 768)
(27, 768)
(60, 768)
(9, 768)
(20, 768)
(9, 768)
(16, 768)
(43, 768)
(17, 768)
(15, 768)
(93, 768)
(69, 768)
(23, 768)
(50, 768)
(36, 768)
(7, 768)
(13, 768)
(94, 768)
(78, 768)
(191, 768)
(98, 768)
(25, 768)
(132, 768)
(34, 768)
(41, 768)
(78, 768)
(10, 768)
(8, 768)
(99, 768)
(56, 768)
(54, 768)
(26, 768)
(74, 768)
(50, 768)
(40, 768)
(26, 768)
(50, 768)
(24, 768)
(32, 768)
(47, 768)
(29, 768)
(80, 768)
(17, 768)
(8, 768)
(200, 768)
(155, 768)
(59, 768)
(56, 768)
(25, 768)
(40, 768)
(122, 768)
(86, 768)
(72, 768)
(92, 768)
(16, 768)
(18, 768)
(29, 768)
(67, 768)
(63, 768)
(54, 768)
(62, 768)
(32, 768)
(37, 768)
(101, 768)
(63, 768)
(47, 768)
(53, 768)
(64, 768)
(90, 768)
(19, 768)
(77, 768)
(11, 768)
(79, 768)
(143, 768)
(67, 768)
(64, 7

In [270]:
from scipy.linalg import lapack
import math
def graph_mini_batch(adj_matrix_list,  x_list,  y_list, batch_size=64):
    assert(len(adj_matrix_list)==len(x_list) and len(x_list)==len(y_list))
    length_left=len(x_list)
    iters_needed = math.ceil(length_left/batch_size)

    for i in range(iters_needed):
        if length_left>=batch_size:
            this_batch=batch_size
        else:
            this_batch=length_left
        length_left -= this_batch
        #this_batch tells us how many things we're going to stick together now
        start_index = i*batch_size
        adj_round = adj_matrix_list[start_index:start_index+this_batch]
        x_round = x_list[start_index:start_index+this_batch]
        y_round = y_list[start_index:start_index+this_batch] #blah_round vars hold list of relevant tensors for this round of iter
        sizes = [x.shape[0] for x in x_round]
        batch_bits = []
        for j in range(this_batch):
            left_sum = sum(sizes[0:j])
            right_sum= sum(sizes[j+1:this_batch])
            tens = adj_round[j] # tens is the tensor at position j to stick bits onto the ends of
            tens_size = tens.shape[0]
            #MATHS: need dim0 to refer to size of tens, dim1 to refer to left and right sum
            left_block = torch.zeros(tens_size,left_sum)
            right_block = torch.zeros(tens_size,right_sum)
#            print(f"left dim: {(tens_size,left_sum)}, right dim: {(tens_size,right_sum)}, middle dim: {tens_size}, block dim: {tens_size+left_sum+right_sum}")
            adj_round[j] = torch.cat((left_block,tens,right_block),dim=1)
            batch_bit = torch.ones(tens_size)
            batch_bit = torch.mul(batch_bit,j+1)
            batch_bits.append(batch_bit)
        #stuck stuff onto adj_round tensors
        A_B = torch.cat(adj_round)
        X_B = torch.cat(x_round)
        Y_B = torch.cat(y_round)
        Batch = torch.cat(batch_bits)
        yield (A_B,X_B,Y_B,Batch)

     # Implement the function here

In [271]:
# Test your batch function here 
for adj_matrix, x, y, batch in graph_mini_batch(A, X, Y, batch_size):
     print(adj_matrix.size())
     print(x.size())
     print(y.size())
     print(batch.size())

torch.Size([189, 189])
torch.Size([189, 768])
torch.Size([4])
torch.Size([189])
torch.Size([284, 284])
torch.Size([284, 768])
torch.Size([4])
torch.Size([284])
torch.Size([268, 268])
torch.Size([268, 768])
torch.Size([4])
torch.Size([268])
torch.Size([250, 250])
torch.Size([250, 768])
torch.Size([4])
torch.Size([250])
torch.Size([66, 66])
torch.Size([66, 768])
torch.Size([4])
torch.Size([66])
torch.Size([161, 161])
torch.Size([161, 768])
torch.Size([4])
torch.Size([161])
torch.Size([166, 166])
torch.Size([166, 768])
torch.Size([4])
torch.Size([166])
torch.Size([88, 88])
torch.Size([88, 768])
torch.Size([4])
torch.Size([88])
torch.Size([194, 194])
torch.Size([194, 768])
torch.Size([4])
torch.Size([194])
torch.Size([116, 116])
torch.Size([116, 768])
torch.Size([4])
torch.Size([116])
torch.Size([376, 376])
torch.Size([376, 768])
torch.Size([4])
torch.Size([376])
torch.Size([289, 289])
torch.Size([289, 768])
torch.Size([4])
torch.Size([289])
torch.Size([137, 137])
torch.Size([137, 768])
to

In [272]:
def global_sum_pool(x, batch):
    return scatter(x,batch.to(torch.int64).add(-1),0)
""" batch_size = int(torch.max(batch).item())
    index = batch.to(torch.int64).add(-1)
    index = torch.reshape(index,(index.shape[0],1))
    index = torch.cat([index]*7,1)
    size = (batch_size,x.shape[1])
    writeTensor = torch.zeros(size)
    writeTensor.scatter(0,index,x)
    return writeTensor"""
   # Implement the function here


' batch_size = int(torch.max(batch).item())\n    index = batch.to(torch.int64).add(-1)\n    index = torch.reshape(index,(index.shape[0],1))\n    index = torch.cat([index]*7,1)\n    size = (batch_size,x.shape[1])\n    writeTensor = torch.zeros(size)\n    writeTensor.scatter(0,index,x)\n    return writeTensor'

In [273]:
# Test your pooling function, assuming you are given a mini-batch of node features and a batch vector
for _, x, _ , batch in graph_mini_batch(A, X, Y, batch_size):
      sum_graph_rep =  global_sum_pool(x, batch)
      print(sum_graph_rep.size())

torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4

In [274]:
class GINConv(MessagePassing):
    def __init__(self, emb_dim):
        '''
            emb_dim (int): node embedding dimensionality
        '''
        super(GINConv, self).__init__(aggr = "add")

        self.mlp = torch.nn.Sequential(torch.nn.Linear(emb_dim, 2*emb_dim), torch.nn.BatchNorm1d(2*emb_dim), torch.nn.ReLU(), torch.nn.Linear(2*emb_dim, emb_dim))
        self.eps = torch.nn.Parameter(torch.Tensor([0]))

    def forward(self, x, edge_index):
        out = self.mlp((1 + self.eps) *x + self.propagate(edge_index, x=x))
        return out

    def message(self, x_j):
        return F.relu(x_j)

    def update(self, aggr_out):
        return aggr_out


### GNN to generate node embedding
class GIN2(torch.nn.Module):
    """
    Output:
        node representations
    """
    def __init__(self, num_layer, emb_dim, hidden_dim, drop_ratio = 0, JK = "last", residual = False):
        '''
            emb_dim (int): node embedding dimensionality
            num_layer (int): number of GNN message passing layers
        '''
        super(GIN, self).__init__()
        self.num_layer = num_layer
        self.drop_ratio = drop_ratio
        self.JK = JK
        ### add residual connection or not
        self.residual = residual
        if self.num_layer < 2:
            raise ValueError("Number of GNN layers must be greater than 1.")

        self.embed = torch.nn.Linear(emb_dim, hidden_dim)
        self.convs = torch.nn.ModuleList()
        self.batch_norms = torch.nn.ModuleList()
        for layer in range(num_layer):
            self.convs.append(GINConv(hidden_dim))
            self.batch_norms.append(torch.nn.BatchNorm1d(hidden_dim))

    def forward(self, x, edge_index):
        h_list = [self.embed(x)]
        for layer in range(self.num_layer):
            h = self.convs[layer](h_list[layer], edge_index)
            h = self.batch_norms[layer](h)
            if layer == self.num_layer - 1:
                #remove relu for the last layer
                h = F.dropout(h, self.drop_ratio, training = self.training)
            else:
                h = F.dropout(F.relu(h), self.drop_ratio, training = self.training)

            if self.residual:
                h += h_list[layer]

            h_list.append(h)
        ### Different implementations of Jk-concat
        if self.JK == "last":
            node_representation = h_list[-1]
        elif self.JK == "sum":
            node_representation = 0
            for layer in range(self.num_layer + 1):
                node_representation += h_list[layer]

        return node_representation

In [275]:
class GCN(nn.Module):
  def __init__(
      self,
      input_dim: int,
      hid_dim: int,
#      n_classes: int,
      n_layers: int,
      dropout_ratio: float = 0.3):
    super(GCN, self).__init__()

    """
    Args:
      input_dim: input feature dimension
      hid_dim: hidden feature dimension
      n_classes: number of target classes
      n_layers: number of layers
      dropout_ratio: dropout_ratio
    """
    ## ------ Begin Solution ------ ##
    self.n_layers = n_layers
    if n_layers == 0:
        self.model = nn.ModuleList(
            [nn.Linear(input_dim,hid_dim),nn.ReLU(),nn.Dropout(dropout_ratio)])
        #assuming we just do a single-layer network
    elif n_layers == 1:
        self.model = nn.ModuleList(
            [GCNConv(input_dim,hid_dim),nn.ReLU(),nn.Dropout(dropout_ratio)])
    else:
        self.model = nn.ModuleList([GCNConv(input_dim,hid_dim),nn.ReLU(), nn.Dropout(dropout_ratio)]+
                              [GCNConv(hid_dim,hid_dim),nn.ReLU(),nn.Dropout(dropout_ratio)]*(n_layers-1))

    ## ------ End Solution ------ ##

  def forward(self, X, A) -> torch.Tensor:
    ## ------ Begin Solution ------ ##
    b = self.generate_node_embeddings(X,A)
    classifier = self.model[-1]
    b = classifier.forward(b)
    return b
    ## ------ End Solution ------ ##

  def generate_node_embeddings(self, X, A) -> torch.Tensor:

    b = X
    for i,l in enumerate(self.model):
        #print(i)#sanity checking

        if self.n_layers == 0 and i >= 3:#break pre-classification in 0-layer case
            break

        if i >= 3*self.n_layers and self.n_layers>0:#break pre classification
            break

        if i%3==0 and self.n_layers !=0:#if non-ReLU / dropout layer
            assert(type(l)==GCNConv)
            #print(f"b and A shapes: {b.shape}, {A.shape}")
            b = l.forward(b,A)

        else:#ReLU / dropout/n_layer=0
            b = l.forward(b)
            
    return b

  def param_init(self):
    ## ------ Begin Solution ------ ##
    """placeholder for refreshing parameters"""
    print("fired up and ready to serve")
    ## ------ End Solution ------ ##

In [276]:
from torch_scatter import scatter
from torch_geometric.utils import softmax as sparse_softmax

In [277]:
params = {
    "input_features": 768,
    "hidden_features": 50,
    "num_layers": 1,
    "learning_rate": 1e-4,
    "weight_decay": 0,
    "num_epochs": 100,
    "num_classes": 3,
    "batch_size": 63
}


In [281]:
model4

GAT(768, 50, num_layers=1)

In [278]:
begin_time = time.time()

#model = GIN(params["num_layers"], params["input_features"], params["hidden_features"])
#model = GCN(params["input_features"], params["hidden_features"], params["num_layers"], 0)
model4 = GAT( params["input_features"], params["hidden_features"],  params["num_layers"])
#GIN num_layer, emb_dim, hidden_dim, drop_ratio = 0
#GCN input_dim: int, hid_dim: int, n_classes: int, n_layers: int, dropout_ratio
#GAT in_channels: int, hidden_channels: int, num_layers: int,

pooling = global_sum_pool
graph_pred_linear = torch.nn.Linear(params["hidden_features"], params["num_classes"])
loss_fn = nn.CrossEntropyLoss()
model_param_group = [{"params": model4.parameters(), "lr": params["learning_rate"]}]
if graph_pred_linear is not None:
        model_param_group.append(
            {"params": graph_pred_linear.parameters(), "lr": params["learning_rate"]}
        )

optimizer = torch.optim.AdamW(model_param_group,
                                lr=params["learning_rate"],
                                weight_decay=params["weight_decay"])
for epoch in range(params["num_epochs"]):
  model4.train()
  for adj_matrix, x, y, batch in graph_mini_batch(A, X, Y, params["batch_size"]):
      optimizer.zero_grad()
      edge_index = U.dense_to_sparse(adj_matrix)[0]
      nodes = model4(x, edge_index)
      graph_reps = pooling(nodes, batch)
      pred = graph_pred_linear(graph_reps)
      print(pred.shape)
      print(y.shape)
      loss = loss_fn(pred, y)
      loss.backward()
      optimizer.step()
  if epoch % 10 == 0:
      model4.eval()
      correct = 0
      total_num = 0 
      for adj_matrix, x, y, batch in graph_mini_batch(A, X, Y, params["batch_size"]):
        optimizer.zero_grad()
        edge_index = U.dense_to_sparse(adj_matrix)[0]
        nodes = model4(x, edge_index)
        graph_reps = pooling(nodes, batch)
        pred = graph_pred_linear(graph_reps)
        correct += (pred.argmax(dim=-1) == y).sum()
        total_num += len(y)
      print("epoch={}, loss={}, accuracy={}".format(epoch, loss.item(), correct/total_num))
print("time={}".format(time.time()-begin_time))

torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([22, 3])
torch.Size([22])
epoch=0, loss=8.179327011108398, accuracy=0.3174999952316284
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([22, 3])
torch.Size([22])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([22, 3])
torch.Size([22])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63

epoch=30, loss=0.3559767007827759, accuracy=0.6600000262260437
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([22, 3])
torch.Size([22])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([22, 3])
torch.Size([22])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([22, 3])
torch.Size([22])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([

torch.Size([63, 3])
torch.Size([63])
torch.Size([22, 3])
torch.Size([22])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([22, 3])
torch.Size([22])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([22, 3])
torch.Size([22])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([22, 3])
torch.Size([22])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
t

torch.Size([63, 3])
torch.Size([63])
torch.Size([22, 3])
torch.Size([22])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([22, 3])
torch.Size([22])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([22, 3])
torch.Size([22])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([22, 3])
torch.Size([22])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
torch.Size([63, 3])
torch.Size([63])
t

In [279]:
begin_time = time.time()

#model = GIN(params["num_layers"], params["input_features"], params["hidden_features"])
model5 = GCN(params["input_features"], params["hidden_features"], params["num_layers"], 0)
#model = GAT( params["input_features"], params["hidden_features"],  params["num_layers"])
#GIN num_layer, emb_dim, hidden_dim, drop_ratio = 0
#GCN input_dim: int, hid_dim: int, n_classes: int, n_layers: int, dropout_ratio
#GAT in_channels: int, hidden_channels: int, num_layers: int,

pooling = global_sum_pool
graph_pred_linear = torch.nn.Linear(params["hidden_features"], params["num_classes"])
loss_fn = nn.CrossEntropyLoss()
model_param_group = [{"params": model5.parameters(), "lr": params["learning_rate"]}]
if graph_pred_linear is not None:
        model_param_group.append(
            {"params": graph_pred_linear.parameters(), "lr": params["learning_rate"]}
        )

optimizer = torch.optim.AdamW(model_param_group,
                                lr=params["learning_rate"],
                                weight_decay=params["weight_decay"])
for epoch in range(params["num_epochs"]):
  model5.train()
  for adj_matrix, x, y, batch in graph_mini_batch(A, X, Y, params["batch_size"]):
      optimizer.zero_grad()
      edge_index = U.dense_to_sparse(adj_matrix)[0]
      nodes = model5(x, edge_index)
      graph_reps = pooling(nodes, batch)
      pred = graph_pred_linear(graph_reps)
      loss = loss_fn(pred, y)
      loss.backward()
      optimizer.step()
  if epoch % 10 == 0:
      model5.eval()
      correct = 0
      total_num = 0 
      for adj_matrix, x, y, batch in graph_mini_batch(A, X, Y, params["batch_size"]):
        optimizer.zero_grad()
        edge_index = U.dense_to_sparse(adj_matrix)[0]
        nodes = model5(x, edge_index)
        graph_reps = pooling(nodes, batch)
        pred = graph_pred_linear(graph_reps)
        correct += (pred.argmax(dim=-1) == y).sum()
        total_num += len(y)
      print("epoch={}, loss={}, accuracy={}".format(epoch, loss.item(), correct/total_num))
print("time={}".format(time.time()-begin_time))

epoch=0, loss=7.02092170715332, accuracy=0.32249999046325684
epoch=10, loss=1.7543059587478638, accuracy=0.42250001430511475
epoch=20, loss=0.5154033899307251, accuracy=0.5600000023841858
epoch=30, loss=0.2884407639503479, accuracy=0.6600000262260437
epoch=40, loss=0.22059696912765503, accuracy=0.737500011920929
epoch=50, loss=0.17770713567733765, accuracy=0.7950000166893005
epoch=60, loss=0.1511521190404892, accuracy=0.8399999737739563
epoch=70, loss=0.12962689995765686, accuracy=0.8849999904632568
epoch=80, loss=0.10416223853826523, accuracy=0.9049999713897705
epoch=90, loss=0.0854136124253273, accuracy=0.9200000166893005
time=117.59667253494263


In [280]:
begin_time = time.time()

model6 = GIN( params["input_features"],params["hidden_features"], params["num_layers"])
#model = GCN(params["input_features"], params["hidden_features"], params["num_layers"], 0)
#model = GAT( params["input_features"], params["hidden_features"],  params["num_layers"])
#GIN num_layer, emb_dim, hidden_dim, drop_ratio = 0
#GCN input_dim: int, hid_dim: int, n_classes: int, n_layers: int, dropout_ratio
#GAT in_channels: int, hidden_channels: int, num_layers: int,

pooling = global_sum_pool
graph_pred_linear = torch.nn.Linear(params["hidden_features"], params["num_classes"])
loss_fn = nn.CrossEntropyLoss()
model_param_group = [{"params": model6.parameters(), "lr": params["learning_rate"]}]
if graph_pred_linear is not None:
        model_param_group.append(
            {"params": graph_pred_linear.parameters(), "lr": params["learning_rate"]}
        )

optimizer = torch.optim.AdamW(model_param_group,
                                lr=params["learning_rate"],
                                weight_decay=params["weight_decay"])
for epoch in range(params["num_epochs"]):
  model6.train()
  for adj_matrix, x, y, batch in graph_mini_batch(A, X, Y, params["batch_size"]):
      optimizer.zero_grad()
      edge_index = U.dense_to_sparse(adj_matrix)[0]
      nodes = model6(x, edge_index)
      graph_reps = pooling(nodes, batch)
      pred = graph_pred_linear(graph_reps)
      loss = loss_fn(pred, y)
      loss.backward()
      optimizer.step()
  if epoch % 10 == 0:
      model6.eval()
      correct = 0
      total_num = 0 
      for adj_matrix, x, y, batch in graph_mini_batch(A, X, Y, params["batch_size"]):
        optimizer.zero_grad()
        edge_index = U.dense_to_sparse(adj_matrix)[0]
        nodes = model6(x, edge_index)
        graph_reps = pooling(nodes, batch)
        pred = graph_pred_linear(graph_reps)
        correct += (pred.argmax(dim=-1) == y).sum()
        total_num += len(y)
      print("epoch={}, loss={}, accuracy={}".format(epoch, loss.item(), correct/total_num))
print("time={}".format(time.time()-begin_time))

epoch=0, loss=83.18081665039062, accuracy=0.35249999165534973
epoch=10, loss=8.627410888671875, accuracy=0.5774999856948853
epoch=20, loss=3.1636321544647217, accuracy=0.6675000190734863
epoch=30, loss=0.7563055157661438, accuracy=0.6524999737739563
epoch=40, loss=0.8476285934448242, accuracy=0.7099999785423279
epoch=50, loss=0.0889531597495079, accuracy=0.7950000166893005
epoch=60, loss=0.03973449021577835, accuracy=0.824999988079071
epoch=70, loss=0.06400588154792786, accuracy=0.8725000023841858
epoch=80, loss=0.02159566432237625, accuracy=0.8974999785423279
epoch=90, loss=0.029274815693497658, accuracy=0.9300000071525574
time=345.0575301647186


In [246]:
model3

GIN(768, 50, num_layers=6)

2D second

In [218]:
# please **DO NOT** modify  any part of the following code  in this cell 
batch_size = 4
A = []
X = []
Y = []
for idx in X_train_2D.index:
    X_shape = X_train_2D[idx][0].shape[0]
    adj_matrix = torch.ones((X_shape,X_shape))
    A.append(adj_matrix)
    X.append(X_train_2D[idx][0])
    Y.append(torch.Tensor([y_train[idx]//2]).long())

In [219]:
print(A[0]) #ADJ matrices of varying size
print(X[0]) #node features (constant width of 7)
print(Y[0]) #graph classification (constant width 1)

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])
tensor([[-0.3308,  1.2500, -0.3503,  ...,  0.8419,  0.6621,  1.4007],
        [-1.2047, -0.7449, -0.4021,  ...,  0.9890,  0.6646,  0.7662],
        [-0.7391,  0.8704,  0.2018,  ...,  0.8420,  0.1410,  1.3532],
        ...,
        [-0.9116, -0.3945, -0.4765,  ...,  0.7293,  0.4933,  1.2296],
        [-0.4236, -1.5236, -0.0414,  ...,  1.3328,  0.8098,  0.7013],
        [ 0.4253, -0.6077, -0.8224,  ...,  1.5106, -0.5938,  0.8994]])
tensor([2])


In [220]:
for i in X:
  print((i.shape[0],i.shape[1]))

(52, 768)
(26, 768)
(62, 768)
(81, 768)
(130, 768)
(23, 768)
(99, 768)
(70, 768)
(33, 768)
(23, 768)
(292, 768)
(34, 768)
(119, 768)
(53, 768)
(114, 768)
(18, 768)
(15, 768)
(12, 768)
(43, 768)
(12, 768)
(70, 768)
(24, 768)
(64, 768)
(37, 768)
(83, 768)
(34, 768)
(67, 768)
(12, 768)
(21, 768)
(12, 768)
(17, 768)
(50, 768)
(20, 768)
(18, 768)
(114, 768)
(80, 768)
(28, 768)
(65, 768)
(51, 768)
(10, 768)
(16, 768)
(107, 768)
(89, 768)
(216, 768)
(147, 768)
(28, 768)
(151, 768)
(39, 768)
(58, 768)
(93, 768)
(11, 768)
(11, 768)
(114, 768)
(65, 768)
(63, 768)
(31, 768)
(83, 768)
(57, 768)
(45, 768)
(33, 768)
(59, 768)
(29, 768)
(35, 768)
(58, 768)
(38, 768)
(91, 768)
(24, 768)
(11, 768)
(231, 768)
(190, 768)
(68, 768)
(67, 768)
(34, 768)
(47, 768)
(141, 768)
(97, 768)
(81, 768)
(113, 768)
(19, 768)
(21, 768)
(36, 768)
(78, 768)
(72, 768)
(63, 768)
(71, 768)
(37, 768)
(46, 768)
(112, 768)
(80, 768)
(58, 768)
(66, 768)
(73, 768)
(103, 768)
(24, 768)
(94, 768)
(14, 768)
(100, 768)
(160, 768)
(7

In [221]:
from scipy.linalg import lapack
import math
def graph_mini_batch(adj_matrix_list,  x_list,  y_list, batch_size=64):
    assert(len(adj_matrix_list)==len(x_list) and len(x_list)==len(y_list))
    length_left=len(x_list)
    iters_needed = math.ceil(length_left/batch_size)

    for i in range(iters_needed):
        if length_left>=batch_size:
            this_batch=batch_size
        else:
            this_batch=length_left
        length_left -= this_batch
        #this_batch tells us how many things we're going to stick together now
        start_index = i*batch_size
        adj_round = adj_matrix_list[start_index:start_index+this_batch]
        x_round = x_list[start_index:start_index+this_batch]
        y_round = y_list[start_index:start_index+this_batch] #blah_round vars hold list of relevant tensors for this round of iter
        sizes = [x.shape[0] for x in x_round]
        batch_bits = []
        for j in range(this_batch):
            left_sum = sum(sizes[0:j])
            right_sum= sum(sizes[j+1:this_batch])
            tens = adj_round[j] # tens is the tensor at position j to stick bits onto the ends of
            tens_size = tens.shape[0]
            #MATHS: need dim0 to refer to size of tens, dim1 to refer to left and right sum
            left_block = torch.zeros(tens_size,left_sum)
            right_block = torch.zeros(tens_size,right_sum)
#            print(f"left dim: {(tens_size,left_sum)}, right dim: {(tens_size,right_sum)}, middle dim: {tens_size}, block dim: {tens_size+left_sum+right_sum}")
            adj_round[j] = torch.cat((left_block,tens,right_block),dim=1)
            batch_bit = torch.ones(tens_size)
            batch_bit = torch.mul(batch_bit,j+1)
            batch_bits.append(batch_bit)
        #stuck stuff onto adj_round tensors
        A_B = torch.cat(adj_round)
        X_B = torch.cat(x_round)
        Y_B = torch.cat(y_round)
        Batch = torch.cat(batch_bits)
        yield (A_B,X_B,Y_B,Batch)

     # Implement the function here

In [222]:
# Test your batch function here 
for adj_matrix, x, y, batch in graph_mini_batch(A, X, Y, batch_size):
     print(adj_matrix.size())
     print(x.size())
     print(y.size())
     print(batch.size())

torch.Size([221, 221])
torch.Size([221, 768])
torch.Size([4])
torch.Size([221])
torch.Size([322, 322])
torch.Size([322, 768])
torch.Size([4])
torch.Size([322])
torch.Size([382, 382])
torch.Size([382, 768])
torch.Size([4])
torch.Size([382])
torch.Size([304, 304])
torch.Size([304, 768])
torch.Size([4])
torch.Size([304])
torch.Size([82, 82])
torch.Size([82, 768])
torch.Size([4])
torch.Size([82])
torch.Size([195, 195])
torch.Size([195, 768])
torch.Size([4])
torch.Size([195])
torch.Size([196, 196])
torch.Size([196, 768])
torch.Size([4])
torch.Size([196])
torch.Size([100, 100])
torch.Size([100, 768])
torch.Size([4])
torch.Size([100])
torch.Size([232, 232])
torch.Size([232, 768])
torch.Size([4])
torch.Size([232])
torch.Size([154, 154])
torch.Size([154, 768])
torch.Size([4])
torch.Size([154])
torch.Size([428, 428])
torch.Size([428, 768])
torch.Size([4])
torch.Size([428])
torch.Size([365, 365])
torch.Size([365, 768])
torch.Size([4])
torch.Size([365])
torch.Size([173, 173])
torch.Size([173, 768]

In [223]:
def global_sum_pool(x, batch):
    return scatter(x,batch.to(torch.int64).add(-1),0)
""" batch_size = int(torch.max(batch).item())
    index = batch.to(torch.int64).add(-1)
    index = torch.reshape(index,(index.shape[0],1))
    index = torch.cat([index]*7,1)
    size = (batch_size,x.shape[1])
    writeTensor = torch.zeros(size)
    writeTensor.scatter(0,index,x)
    return writeTensor"""
   # Implement the function here


' batch_size = int(torch.max(batch).item())\n    index = batch.to(torch.int64).add(-1)\n    index = torch.reshape(index,(index.shape[0],1))\n    index = torch.cat([index]*7,1)\n    size = (batch_size,x.shape[1])\n    writeTensor = torch.zeros(size)\n    writeTensor.scatter(0,index,x)\n    return writeTensor'

In [224]:
# Test your pooling function, assuming you are given a mini-batch of node features and a batch vector
for _, x, _ , batch in graph_mini_batch(A, X, Y, batch_size):
      sum_graph_rep =  global_sum_pool(x, batch)
      print(sum_graph_rep.size())

torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([4

In [225]:
class GINConv(MessagePassing):
    def __init__(self, emb_dim):
        '''
            emb_dim (int): node embedding dimensionality
        '''
        super(GINConv, self).__init__(aggr = "add")

        self.mlp = torch.nn.Sequential(torch.nn.Linear(emb_dim, 2*emb_dim), torch.nn.BatchNorm1d(2*emb_dim), torch.nn.ReLU(), torch.nn.Linear(2*emb_dim, emb_dim))
        self.eps = torch.nn.Parameter(torch.Tensor([0]))

    def forward(self, x, edge_index):
        out = self.mlp((1 + self.eps) *x + self.propagate(edge_index, x=x))
        return out

    def message(self, x_j):
        return F.relu(x_j)

    def update(self, aggr_out):
        return aggr_out


### GNN to generate node embedding
class GIN(torch.nn.Module):
    """
    Output:
        node representations
    """
    def __init__(self, num_layer, emb_dim, hidden_dim, drop_ratio = 0, JK = "last", residual = False):
        '''
            emb_dim (int): node embedding dimensionality
            num_layer (int): number of GNN message passing layers
        '''
        super(GIN, self).__init__()
        self.num_layer = num_layer
        self.drop_ratio = drop_ratio
        self.JK = JK
        ### add residual connection or not
        self.residual = residual
        if self.num_layer < 2:
            raise ValueError("Number of GNN layers must be greater than 1.")

        self.embed = torch.nn.Linear(emb_dim, hidden_dim)
        self.convs = torch.nn.ModuleList()
        self.batch_norms = torch.nn.ModuleList()
        for layer in range(num_layer):
            self.convs.append(GINConv(hidden_dim))
            self.batch_norms.append(torch.nn.BatchNorm1d(hidden_dim))

    def forward(self, x, edge_index):
        h_list = [self.embed(x)]
        for layer in range(self.num_layer):
            h = self.convs[layer](h_list[layer], edge_index)
            h = self.batch_norms[layer](h)
            if layer == self.num_layer - 1:
                #remove relu for the last layer
                h = F.dropout(h, self.drop_ratio, training = self.training)
            else:
                h = F.dropout(F.relu(h), self.drop_ratio, training = self.training)

            if self.residual:
                h += h_list[layer]

            h_list.append(h)
        ### Different implementations of Jk-concat
        if self.JK == "last":
            node_representation = h_list[-1]
        elif self.JK == "sum":
            node_representation = 0
            for layer in range(self.num_layer + 1):
                node_representation += h_list[layer]

        return node_representation

In [226]:
class GCN(nn.Module):
  def __init__(
      self,
      input_dim: int,
      hid_dim: int,
#      n_classes: int,
      n_layers: int,
      dropout_ratio: float = 0.3):
    super(GCN, self).__init__()

    """
    Args:
      input_dim: input feature dimension
      hid_dim: hidden feature dimension
      n_classes: number of target classes
      n_layers: number of layers
      dropout_ratio: dropout_ratio
    """
    ## ------ Begin Solution ------ ##
    self.n_layers = n_layers
    if n_layers == 0:
        self.model = nn.ModuleList(
            [nn.Linear(input_dim,hid_dim),nn.ReLU(),nn.Dropout(dropout_ratio)])
        #assuming we just do a single-layer network
    elif n_layers == 1:
        self.model = nn.ModuleList(
            [GCNConv(input_dim,hid_dim),nn.ReLU(),nn.Dropout(dropout_ratio)])
    else:
        self.model = nn.ModuleList([GCNConv(input_dim,hid_dim),nn.ReLU(), nn.Dropout(dropout_ratio)]+
                              [GCNConv(hid_dim,hid_dim),nn.ReLU(),nn.Dropout(dropout_ratio)]*(n_layers-1))

    ## ------ End Solution ------ ##

  def forward(self, X, A) -> torch.Tensor:
    ## ------ Begin Solution ------ ##
    b = self.generate_node_embeddings(X,A)
    classifier = self.model[-1]
    b = classifier.forward(b)
    return b
    ## ------ End Solution ------ ##

  def generate_node_embeddings(self, X, A) -> torch.Tensor:

    b = X
    for i,l in enumerate(self.model):
        #print(i)#sanity checking

        if self.n_layers == 0 and i >= 3:#break pre-classification in 0-layer case
            break

        if i >= 3*self.n_layers and self.n_layers>0:#break pre classification
            break

        if i%3==0 and self.n_layers !=0:#if non-ReLU / dropout layer
            assert(type(l)==GCNConv)
            #print(f"b and A shapes: {b.shape}, {A.shape}")
            b = l.forward(b,A)

        else:#ReLU / dropout/n_layer=0
            b = l.forward(b)
            
    return b

  def param_init(self):
    ## ------ Begin Solution ------ ##
    """placeholder for refreshing parameters"""
    print("fired up and ready to serve")
    ## ------ End Solution ------ ##

In [227]:
from torch_scatter import scatter
from torch_geometric.utils import softmax as sparse_softmax

In [228]:
params = {
    "input_features": 768,
    "hidden_features": 50,
    "num_layers": 3,
    "learning_rate": 1e-4,
    "weight_decay": 0,
    "num_epochs": 100,
    "num_classes": 3,
    "batch_size": 63
}


In [229]:
begin_time = time.time()

#model = GIN(params["num_layers"], params["input_features"], params["hidden_features"])
#model = GCN(params["input_features"], params["hidden_features"], params["num_layers"], 0)
model = GAT( params["input_features"], params["hidden_features"],  params["num_layers"])
#GIN num_layer, emb_dim, hidden_dim, drop_ratio = 0
#GCN input_dim: int, hid_dim: int, n_classes: int, n_layers: int, dropout_ratio
#GAT in_channels: int, hidden_channels: int, num_layers: int,

pooling = global_sum_pool
graph_pred_linear = torch.nn.Linear(params["hidden_features"], params["num_classes"])
loss_fn = nn.CrossEntropyLoss()
model_param_group = [{"params": model.parameters(), "lr": params["learning_rate"]}]
if graph_pred_linear is not None:
        model_param_group.append(
            {"params": graph_pred_linear.parameters(), "lr": params["learning_rate"]}
        )

optimizer = torch.optim.AdamW(model_param_group,
                                lr=params["learning_rate"],
                                weight_decay=params["weight_decay"])
for epoch in range(params["num_epochs"]):
  model.train()
  for adj_matrix, x, y, batch in graph_mini_batch(A, X, Y, params["batch_size"]):
      optimizer.zero_grad()
      edge_index = U.dense_to_sparse(adj_matrix)[0]
      nodes = model(x, edge_index)
      graph_reps = pooling(nodes, batch)
      pred = graph_pred_linear(graph_reps)
      loss = loss_fn(pred, y)
      loss.backward()
      optimizer.step()
  if epoch % 10 == 0:
      model.eval()
      correct = 0
      total_num = 0 
      for adj_matrix, x, y, batch in graph_mini_batch(A, X, Y, params["batch_size"]):
        optimizer.zero_grad()
        edge_index = U.dense_to_sparse(adj_matrix)[0]
        nodes = model(x, edge_index)
        graph_reps = pooling(nodes, batch)
        pred = graph_pred_linear(graph_reps)
        correct += (pred.argmax(dim=-1) == y).sum()
        total_num += len(y)
      print("epoch={}, loss={}, accuracy={}".format(epoch, loss.item(), correct/total_num))
print("time={}".format(time.time()-begin_time))

epoch=0, loss=3.084766387939453, accuracy=0.35749998688697815
epoch=10, loss=0.7143516540527344, accuracy=0.5274999737739563
epoch=20, loss=0.46007871627807617, accuracy=0.6000000238418579
epoch=30, loss=0.33601513504981995, accuracy=0.699999988079071
epoch=40, loss=0.2637840211391449, accuracy=0.7574999928474426
epoch=50, loss=0.21299123764038086, accuracy=0.8174999952316284
epoch=60, loss=0.17073434591293335, accuracy=0.862500011920929
epoch=70, loss=0.1350405365228653, accuracy=0.8799999952316284
epoch=80, loss=0.10529328137636185, accuracy=0.9049999713897705
epoch=90, loss=0.0828973576426506, accuracy=0.9325000047683716
time=640.5700714588165


In [230]:
begin_time = time.time()

#model = GIN(params["num_layers"], params["input_features"], params["hidden_features"])
model = GCN(params["input_features"], params["hidden_features"], params["num_layers"], 0)
#model = GAT( params["input_features"], params["hidden_features"],  params["num_layers"])
#GIN num_layer, emb_dim, hidden_dim, drop_ratio = 0
#GCN input_dim: int, hid_dim: int, n_classes: int, n_layers: int, dropout_ratio
#GAT in_channels: int, hidden_channels: int, num_layers: int,

pooling = global_sum_pool
graph_pred_linear = torch.nn.Linear(params["hidden_features"], params["num_classes"])
loss_fn = nn.CrossEntropyLoss()
model_param_group = [{"params": model.parameters(), "lr": params["learning_rate"]}]
if graph_pred_linear is not None:
        model_param_group.append(
            {"params": graph_pred_linear.parameters(), "lr": params["learning_rate"]}
        )

optimizer = torch.optim.AdamW(model_param_group,
                                lr=params["learning_rate"],
                                weight_decay=params["weight_decay"])
for epoch in range(params["num_epochs"]):
  model.train()
  for adj_matrix, x, y, batch in graph_mini_batch(A, X, Y, params["batch_size"]):
      optimizer.zero_grad()
      edge_index = U.dense_to_sparse(adj_matrix)[0]
      nodes = model(x, edge_index)
      graph_reps = pooling(nodes, batch)
      pred = graph_pred_linear(graph_reps)
      loss = loss_fn(pred, y)
      loss.backward()
      optimizer.step()
  if epoch % 10 == 0:
      model.eval()
      correct = 0
      total_num = 0 
      for adj_matrix, x, y, batch in graph_mini_batch(A, X, Y, params["batch_size"]):
        optimizer.zero_grad()
        edge_index = U.dense_to_sparse(adj_matrix)[0]
        nodes = model(x, edge_index)
        graph_reps = pooling(nodes, batch)
        pred = graph_pred_linear(graph_reps)
        correct += (pred.argmax(dim=-1) == y).sum()
        total_num += len(y)
      print("epoch={}, loss={}, accuracy={}".format(epoch, loss.item(), correct/total_num))
print("time={}".format(time.time()-begin_time))

epoch=0, loss=8.440476417541504, accuracy=0.35249999165534973
epoch=10, loss=0.9311999678611755, accuracy=0.4625000059604645
epoch=20, loss=0.6574505567550659, accuracy=0.5899999737739563
epoch=30, loss=0.490168958902359, accuracy=0.6825000047683716
epoch=40, loss=0.40397167205810547, accuracy=0.7400000095367432
epoch=50, loss=0.33988574147224426, accuracy=0.7900000214576721
epoch=60, loss=0.2869752049446106, accuracy=0.8149999976158142
epoch=70, loss=0.24919123947620392, accuracy=0.8349999785423279
epoch=80, loss=0.21391314268112183, accuracy=0.8575000166893005
epoch=90, loss=0.18527907133102417, accuracy=0.8899999856948853
time=342.88436794281006


In [231]:
begin_time = time.time()

model = GIN(params["num_layers"], params["input_features"], params["hidden_features"])
#model = GCN(params["input_features"], params["hidden_features"], params["num_layers"], 0)
#model = GAT( params["input_features"], params["hidden_features"],  params["num_layers"])
#GIN num_layer, emb_dim, hidden_dim, drop_ratio = 0
#GCN input_dim: int, hid_dim: int, n_classes: int, n_layers: int, dropout_ratio
#GAT in_channels: int, hidden_channels: int, num_layers: int,

pooling = global_sum_pool
graph_pred_linear = torch.nn.Linear(params["hidden_features"], params["num_classes"])
loss_fn = nn.CrossEntropyLoss()
model_param_group = [{"params": model.parameters(), "lr": params["learning_rate"]}]
if graph_pred_linear is not None:
        model_param_group.append(
            {"params": graph_pred_linear.parameters(), "lr": params["learning_rate"]}
        )

optimizer = torch.optim.AdamW(model_param_group,
                                lr=params["learning_rate"],
                                weight_decay=params["weight_decay"])
for epoch in range(params["num_epochs"]):
  model.train()
  for adj_matrix, x, y, batch in graph_mini_batch(A, X, Y, params["batch_size"]):
      optimizer.zero_grad()
      edge_index = U.dense_to_sparse(adj_matrix)[0]
      nodes = model(x, edge_index)
      graph_reps = pooling(nodes, batch)
      pred = graph_pred_linear(graph_reps)
      loss = loss_fn(pred, y)
      loss.backward()
      optimizer.step()
  if epoch % 10 == 0:
      model.eval()
      correct = 0
      total_num = 0 
      for adj_matrix, x, y, batch in graph_mini_batch(A, X, Y, params["batch_size"]):
        optimizer.zero_grad()
        edge_index = U.dense_to_sparse(adj_matrix)[0]
        nodes = model(x, edge_index)
        graph_reps = pooling(nodes, batch)
        pred = graph_pred_linear(graph_reps)
        correct += (pred.argmax(dim=-1) == y).sum()
        total_num += len(y)
      print("epoch={}, loss={}, accuracy={}".format(epoch, loss.item(), correct/total_num))
print("time={}".format(time.time()-begin_time))

epoch=0, loss=16.444997787475586, accuracy=0.35749998688697815
epoch=10, loss=0.8100470900535583, accuracy=0.38749998807907104
epoch=20, loss=0.2421075403690338, accuracy=0.39250001311302185
epoch=30, loss=0.18983203172683716, accuracy=0.4124999940395355
epoch=40, loss=0.16834530234336853, accuracy=0.4074999988079071
epoch=50, loss=0.1504245400428772, accuracy=0.4025000035762787
epoch=60, loss=0.13843676447868347, accuracy=0.4099999964237213
epoch=70, loss=0.1278468370437622, accuracy=0.4099999964237213
epoch=80, loss=0.1270938366651535, accuracy=0.39750000834465027
epoch=90, loss=2.5207064151763916, accuracy=0.4000000059604645
time=357.6910216808319


In [248]:
def sym_norm_adj(A):
    #### Create the symmetric normalised adjacency from the dense adj matrix A
    # This should return a sparse adjacency matrix. (torch sparse coo tensor format)
    D = torch.diag(torch.sum(A,1))
    A_tilde = A + torch.eye(A.shape[0])
    D_tilde = D + torch.eye(A.shape[0])
#    D_tilde_inv_sqrt = D.apply_(func)
    D_tilde_inv_sqrt = torch.pow(D_tilde,-0.5)
    D_tilde_inv_sqrt[torch.isinf(D_tilde_inv_sqrt)] = 0.0
    A_tilde = A_tilde.to_sparse()
    D_tilde_inv_sqrt = D_tilde_inv_sqrt.to_sparse()
    adj_norm = torch.sparse.mm(torch.sparse.mm(D_tilde_inv_sqrt,A_tilde),D_tilde_inv_sqrt)

    return adj_norm

In [247]:
def dirichlet_energy(X, adj_norm):
    # X is a matrix of shape (num_nodes, feature_channels)
    # adj_norm is a torch sparse coo tensor 
    ######## Your code here ##############
    # print(X.shape)
    # x = X.detach()
    # adj = adj_norm.to_dense()
    # adj = torch.eye(adj.shape[0])-adj
    # energy = torch.trace(torch.matmul(x.T,torch.matmul(adj,x)))

    X = X.detach()
    LX = X - torch.sparse.mm(adj_norm, X)
    energy = (X*LX).sum()
    
    """sp_X = x.to_sparse()
    sp_X_T = x.T.to_sparse()
    adj = adj_norm.to_dense()
    Delta = torch.eye(adj.shape[0]) - adj
    #DELTA  = I - A tilde norm
    energy = torch.trace( torch.sparse.mm( torch.sparse.mm( sp_X_T, Delta.to_sparse() ), sp_X ).to_dense() )"""
    ######################################
    return energy

In [282]:
# please **DO NOT** modify  any part of the following code  in this cell 
batch_size = 1
A = []
X = []
Y = []
for idx in X_test_1D.index:
    X_shape = X_test_1D[idx][0].shape[0]
    adj_matrix = torch.ones((X_shape,X_shape))
    A.append(adj_matrix)
    X.append(X_test_1D[idx][0])
    Y.append(torch.Tensor([y_test[idx]//2]).long())

In [283]:
print(A[0]) #ADJ matrices of varying size
print(X[0]) #node features (constant width of 7)
print(Y[0]) #graph classification (constant width 1)

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])
tensor([[-0.1291,  0.9871,  0.2443,  ...,  0.9427,  0.7077,  1.1700],
        [ 0.7835, -0.2454,  2.0693,  ...,  1.0465,  0.9835,  1.0081],
        [ 1.0474, -1.2752,  1.7462,  ...,  0.8983,  0.4725,  1.3472],
        ...,
        [ 0.2070,  1.0123, -0.2538,  ...,  1.2943,  0.1332,  1.0755],
        [ 0.9213, -0.6303, -0.5782,  ...,  1.5014,  0.6103,  0.3140],
        [ 0.9705, -0.6151,  1.3825,  ...,  1.2134,  0.4370,  0.7035]])
tensor([2])


In [284]:
for i in X:
  print((i.shape[0],i.shape[1]))

(85, 768)
(16, 768)
(14, 768)
(75, 768)
(62, 768)
(15, 768)
(49, 768)
(35, 768)
(104, 768)
(17, 768)
(39, 768)
(98, 768)
(51, 768)
(11, 768)
(56, 768)
(39, 768)
(30, 768)
(83, 768)
(30, 768)
(81, 768)
(16, 768)
(83, 768)
(25, 768)
(85, 768)
(63, 768)
(171, 768)
(42, 768)
(31, 768)
(6, 768)
(154, 768)
(9, 768)
(83, 768)
(66, 768)
(21, 768)
(54, 768)
(141, 768)
(9, 768)
(33, 768)
(110, 768)
(52, 768)
(13, 768)
(54, 768)
(10, 768)
(48, 768)
(30, 768)
(49, 768)
(43, 768)
(56, 768)
(74, 768)
(50, 768)


In [285]:
from scipy.linalg import lapack
import math
def graph_mini_batch(adj_matrix_list,  x_list,  y_list, batch_size=64):
    assert(len(adj_matrix_list)==len(x_list) and len(x_list)==len(y_list))
    length_left=len(x_list)
    iters_needed = math.ceil(length_left/batch_size)

    for i in range(iters_needed):
        if length_left>=batch_size:
            this_batch=batch_size
        else:
            this_batch=length_left
        length_left -= this_batch
        #this_batch tells us how many things we're going to stick together now
        start_index = i*batch_size
        adj_round = adj_matrix_list[start_index:start_index+this_batch]
        x_round = x_list[start_index:start_index+this_batch]
        y_round = y_list[start_index:start_index+this_batch] #blah_round vars hold list of relevant tensors for this round of iter
        sizes = [x.shape[0] for x in x_round]
        batch_bits = []
        for j in range(this_batch):
            left_sum = sum(sizes[0:j])
            right_sum= sum(sizes[j+1:this_batch])
            tens = adj_round[j] # tens is the tensor at position j to stick bits onto the ends of
            tens_size = tens.shape[0]
            #MATHS: need dim0 to refer to size of tens, dim1 to refer to left and right sum
            left_block = torch.zeros(tens_size,left_sum)
            right_block = torch.zeros(tens_size,right_sum)
#            print(f"left dim: {(tens_size,left_sum)}, right dim: {(tens_size,right_sum)}, middle dim: {tens_size}, block dim: {tens_size+left_sum+right_sum}")
            adj_round[j] = torch.cat((left_block,tens,right_block),dim=1)
            batch_bit = torch.ones(tens_size)
            batch_bit = torch.mul(batch_bit,j+1)
            batch_bits.append(batch_bit)
        #stuck stuff onto adj_round tensors
        A_B = torch.cat(adj_round)
        X_B = torch.cat(x_round)
        Y_B = torch.cat(y_round)
        Batch = torch.cat(batch_bits)
        yield (A_B,X_B,Y_B,Batch)

     # Implement the function here

In [286]:
# Test your batch function here 
for adj_matrix, x, y, batch in graph_mini_batch(A, X, Y, batch_size):
     print(adj_matrix.size())
     print(x.size())
     print(y.size())
     print(batch.size())

torch.Size([85, 85])
torch.Size([85, 768])
torch.Size([1])
torch.Size([85])
torch.Size([16, 16])
torch.Size([16, 768])
torch.Size([1])
torch.Size([16])
torch.Size([14, 14])
torch.Size([14, 768])
torch.Size([1])
torch.Size([14])
torch.Size([75, 75])
torch.Size([75, 768])
torch.Size([1])
torch.Size([75])
torch.Size([62, 62])
torch.Size([62, 768])
torch.Size([1])
torch.Size([62])
torch.Size([15, 15])
torch.Size([15, 768])
torch.Size([1])
torch.Size([15])
torch.Size([49, 49])
torch.Size([49, 768])
torch.Size([1])
torch.Size([49])
torch.Size([35, 35])
torch.Size([35, 768])
torch.Size([1])
torch.Size([35])
torch.Size([104, 104])
torch.Size([104, 768])
torch.Size([1])
torch.Size([104])
torch.Size([17, 17])
torch.Size([17, 768])
torch.Size([1])
torch.Size([17])
torch.Size([39, 39])
torch.Size([39, 768])
torch.Size([1])
torch.Size([39])
torch.Size([98, 98])
torch.Size([98, 768])
torch.Size([1])
torch.Size([98])
torch.Size([51, 51])
torch.Size([51, 768])
torch.Size([1])
torch.Size([51])
torch.Si

In [287]:
def global_sum_pool(x, batch):
    return scatter(x,batch.to(torch.int64).add(-1),0)
""" batch_size = int(torch.max(batch).item())
    index = batch.to(torch.int64).add(-1)
    index = torch.reshape(index,(index.shape[0],1))
    index = torch.cat([index]*7,1)
    size = (batch_size,x.shape[1])
    writeTensor = torch.zeros(size)
    writeTensor.scatter(0,index,x)
    return writeTensor"""
   # Implement the function here


' batch_size = int(torch.max(batch).item())\n    index = batch.to(torch.int64).add(-1)\n    index = torch.reshape(index,(index.shape[0],1))\n    index = torch.cat([index]*7,1)\n    size = (batch_size,x.shape[1])\n    writeTensor = torch.zeros(size)\n    writeTensor.scatter(0,index,x)\n    return writeTensor'

In [288]:
# Test your pooling function, assuming you are given a mini-batch of node features and a batch vector
for _, x, _ , batch in graph_mini_batch(A, X, Y, batch_size):
      sum_graph_rep =  global_sum_pool(x, batch)
      print(sum_graph_rep.size())

torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1

dirichlet_energy(X[0],sym_norm_adj(A[0]))

In [259]:
dirichlet_energy(model1(X[0],U.dense_to_sparse(A[0])[0]),sym_norm_adj(A[0]))

tensor(6.7426e-05)

In [260]:
dirichlet_energy(model2(X[0],U.dense_to_sparse(A[0])[0]),sym_norm_adj(A[0]))

tensor(-6.3386e-06)

In [261]:
dirichlet_energy(model3(X[0],U.dense_to_sparse(A[0])[0]),sym_norm_adj(A[0]))

tensor(-1.7607e+09)

In [263]:
model1(X[0], U.dense_to_sparse(A[0])[0]).shape

torch.Size([85, 50])

model1 GAT 6
model2 GCN 6
model3 GIN 6
model4 GAT 1
model5 GCN 1
model6 GIN 1

In [294]:
summ = []
i=0
for a, x, _ , _ in graph_mini_batch(A, X, Y, batch_size):
#     print(a.shape)
#     print(x.shape)
    i+=1
    summ.append([dirichlet_energy(model1(x,U.dense_to_sparse(a)[0]),sym_norm_adj(a)),
                 dirichlet_energy(model2(x,U.dense_to_sparse(a)[0]),sym_norm_adj(a)),
                 dirichlet_energy(model3(x,U.dense_to_sparse(a)[0]),sym_norm_adj(a)),
                 dirichlet_energy(model4(x,U.dense_to_sparse(a)[0]),sym_norm_adj(a)),
                 dirichlet_energy(model5(x,U.dense_to_sparse(a)[0]),sym_norm_adj(a)),
                 dirichlet_energy(model6(x,U.dense_to_sparse(a)[0]),sym_norm_adj(a))])

In [296]:
np.mean(np.array(summ),axis=0)

array([ 1.5213759e-06, -9.2724628e-07, -1.2052107e+12,  3.1134536e+00,
        7.1754635e-05,  4.5141514e+01], dtype=float32)